In [ ]:
from shapely.geometry import Polygon
import pandas as pd

# Import imageloader
import rasterio
import cv2

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual

import os

%matplotlib inline
%reload_ext autoreload

## Read the dataset

In [ ]:
# labels: germany_dataset/labels
# images: germany_dataset/images

# For every image, there is a corresponding label file with the same name

# The labels file is a space separated file with the following columns:
# 1. category
# 2. x-center
# 3. y-center
# 4. x-width
# 5. y-width
def load_image_and_labels(file):
    labels = pd.read_csv('germany_dataset/labels/' + file, sep=' ', header=None)
    labels.columns = ['category', 'x_center', 'y_center', 'x_width', 'y_width']
    
    # Load the image
    image_name = file.replace('txt', 'tif')
    image = cv2.imread('germany_dataset/images/' + image_name)

    # image size = 832 x 832
    # The x-center and y-center are in the range [0, 1]

    # Create a list of polygons
    polygons = []
    for i in range(labels.shape[0]):
        x_center = labels.iloc[i, 1]
        y_center = labels.iloc[i, 2]
        x_width = labels.iloc[i, 3]
        y_width = labels.iloc[i, 4]

        x1 = (x_center - x_width / 2) * 832
        x2 = (x_center + x_width / 2) * 832
        y1 = (y_center - y_width / 2) * 832
        y2 = (y_center + y_width / 2) * 832

        polygons.append(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]))
        
    return image, polygons

In [ ]:
file = os.listdir('germany_dataset/labels')[0]
image, polygons = load_image_and_labels(file)

# Plot the image with the polygons drawn in
def plot_image_and_labels(image, polygons):
    # Convert the image to a numpy array
    image = np.array(image)

    for polygon in polygons:
        x, y = polygon.exterior.xy
        plt.plot(x, y, c='r')
        
    # Plot the image
    plt.imshow(image)
    plt.show()
    
plot_image_and_labels(image, polygons)

In [ ]:
# Convert the polygons to a mask to be used in the model
# The mask will be a 832 x 832 x 1 array
def polygons_to_mask(polygons):
    mask = np.zeros((832, 832, 1))
    for polygon in polygons:
        x, y = polygon.exterior.xy
        x = np.array(x, dtype=np.int32)
        y = np.array(y, dtype=np.int32)
        mask = cv2.fillPoly(mask, [np.column_stack((x, y))], (255))
    return mask

# Convert the mask to a polygon
def mask_to_polygons(mask):
    mask = mask.astype(np.uint8)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        polygons.append(Polygon([(x, y), (x + w, y), (x + w, y + h), (x, y + h)]))
    return polygons

In [ ]:
# Plot the mask and the images in the same plot
mask = polygons_to_mask(polygons)

def plot_image_and_mask(image, mask):
    
    # Convert the image to a numpy array
    image = np.array(image)

    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    
    plt.show()
    
plot_image_and_mask(image, mask)

### Load the dataset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
dataset = os.listdir('germany_dataset/labels')

class ImageLoader(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, polygons = load_image_and_labels(self.dataset[idx])
        mask = polygons_to_mask(polygons)
        
        return torch.tensor(image, dtype=torch.float32).view(3, 832, 832), torch.tensor(mask, dtype=torch.float32).view(1, 832, 832)

In [ ]:
# From the dataset, create a train and test set
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)

train_dataset = ImageLoader(train)
test_dataset = ImageLoader(test)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

### Train the model

In [ ]:
import pytorch_lightning as pl
from models.base import BaseModel
from models.architectures.deep_lab import DeepLabModel
from models.architectures.mask_rcnn import MaskRCNNModel

In [ ]:
model = DeepLabModel(input_size=832, num_classes=1)

# model = MaskRCNNModel(num_classes=2)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

base_model = BaseModel(model, loss_fn, optimizer)
trainer = pl.Trainer(max_epochs=10, min_epochs=5)

In [ ]:
for image, label in train_loader:
    # Print the size of the image and label
    print(image.size(), label.size())  
    break

In [ ]:
trainer.fit(base_model, train_loader, )

In [ ]:
# Estimate the model on the test set
with torch.no_grad():
    for image, label in test_loader:
        output = model(image)
                
        # Convert the output to a polygon
        polygons = mask_to_polygons(output.numpy())
        
        # Plot the image and the polygons
        plot_image_and_labels(file)
        
        break
        